In [1]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
import re
import matplotlib.pyplot as plt
import math
from nltk.corpus import stopwords


In [2]:
Data = pd.read_csv("JobDataTranslated.csv")

In [3]:
# function to preprocess speech
#Data['Job Description'].apply(lambda x: x.lower())

def clean(text):
    
    # removing paragraph numbers
    text = re.sub('[0-9]+.\t','',str(text))
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing hyphens
    text = re.sub("-",' ',str(text))
    text = re.sub("— ",'',str(text))
    # removing quotation marks
    text = re.sub('\"','',str(text))
    # removing salutations
    text = re.sub("Mr\.",'Mr',str(text))
    text = re.sub("Mrs\.",'Mrs',str(text))
    # removing any reference to outside text
    text = re.sub("[\(\[].*?[\)\]]", "", str(text))
    
    return text

# preprocessing speeches
Data['Cleaned Job Description'] = Data['Job Description'].apply(clean)

from string import punctuation

def remove_punctuation(s):
    global punctuation
    for p in punctuation:
        s = s.replace(p, '')
    return s

Data['Cleaned Job Description'] = Data['Cleaned Job Description'].map(remove_punctuation)

Data['Cleaned Job Description'] = Data['Cleaned Job Description'].apply(lambda x: x.lower())


In [6]:
#Data['Job Description'].apply(lambda x: x.lower())

#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
corpus = " ".join(Data['Cleaned Job Description'].tolist())
tokens = word_tokenize(corpus)
# Get the parts of speech tag for all words
answer = nltk.pos_tag(tokens)
answer_pos = [a[1] for a in answer]

# print a value count for the parts of speech
all_pos = pd.Series(answer_pos)
all_pos.value_counts().head()


NN     106773
IN      57169
JJ      45685
DT      40667
NNS     32368
dtype: int64

In [8]:
#nltk.download('stopwords')
en_stopwords = stopwords.words('english')
def remove_stopwords(s):
    global en_stopwords
    s = word_tokenize(s)
    s = " ".join([w for w in s if w not in en_stopwords])
    return s

# Create a new column of descriptions with no stopwords
Data['Cleaned Job Description NoStop'] = Data['Cleaned Job Description'].map(remove_stopwords)

# make a corpus of all the words in the job description
corpus = " ".join(Data['Cleaned Job Description NoStop'].tolist())

# This is the NLTK function that breaks a string down to its tokens
tokens = word_tokenize(corpus)

answer = nltk.pos_tag(tokens)
answer_pos = [a[1] for a in answer]

all_pos = pd.Series(answer_pos)
all_pos.value_counts().head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dima\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


NN     103824
JJ      48363
NNS     31957
VBP     14378
VBG     13640
dtype: int64

In [9]:
# import the necessary functions from the nltk library
#nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

# prepare corpus from the descriptions that dont have stopwords
corpus = " ".join(Data['Cleaned Job Description NoStop'].tolist())

#tokenize words
tokenized_corpus = nltk.word_tokenize(corpus)

# lemmatize these tokens
lemmatized_tokens = [lmtzr.lemmatize(token) for token in tokenized_corpus]

# word frequencies for the lemmatized tokens
fd = nltk.FreqDist(lemmatized_tokens)

# get the top words
top_words = []
for key, value in fd.items():
    top_words.append((key, value))

# sort the list by the top frequencies
top_words = sorted(top_words, key = lambda x:x[1], reverse = True)

# keep top 10 words only



In [25]:

data = pd.read_csv(r"C:\Users\Dima\Downloads\data_merged.csv")
data=pd.DataFrame(data)
#data=data[data["Is requirment"]==1]
indicies = [i for i in range(len(data)) if data["Is requirment"][i]==1]

words = [data['word'][i] for i in indicies]


'service'

In [15]:


def find_word(text, word):
    
    indices = [i for i in range(len(text)) if text[i] == word]
    return indices

def find_words(text,words):
    z=[]
    for i in words:
        if (len(find_word(text,i))!=0):
            z.append(find_word(text,i))
    unlist=[z[i][j] for i in range(len(z)) for j in range(len(z[i]))]
    return unlist

useless=["you","your","and","of","for","are","a","to","the","an","is","also","or","per","in","we","with","our","us","this","on",
        "×","as","who","on"]
# loc - set of indicies with all words in text 
def get_neigbour(text,loc, left=True):
    z=[]
    if (left):
        for i in loc:
            if (i!=0 and useless.count(text[i-1])==0):
                z.append(text[i-1]+" "+text[i])
    else:
        for i in loc:
            if (i!=len(text)-1 and useless.count(text[i+1])==0):
                z.append(text[i]+" "+text[i+1])
            
    return z



req=[]
for i in range(1000):
    txt = Data["Cleaned Job Description"][i]
    x = txt.split()
    new_w=get_neigbour(x,find_words(x,words),True)
    req.append(new_w)

def ugly_list_to_cute_df():
    req_unlist = [req[i][j] for i in range(len(req)) for j in range(len(req[i]))]
    final = list(dict.fromkeys(req_unlist))

    z=[]
    for i in final:
        z.append(req_unlist.count(i))

    datta = {'frequency':  z, 'Combo': final}
    dtf = pd.DataFrame (datta, columns = ['frequency','Combo'])
    dtf=dtf.sort_values(by=['frequency'], ascending=False)
    return dtf

df = ugly_list_to_cute_df()
#df.to_csv("left_neighbours.csv")

In [16]:
def get_neigbours(text,loc):
    z=[]
    for i in loc:
        if (i!=0 and i!=len(text)-1):
            if (useless.count(text[i-1])==0 and useless.count(text[i+1])==0):
                z.append(text[i-1]+" "+text[i]+" "+ text[i+1])
            
    return z

req=[]
for i in range(1000):
    txt = Data["Cleaned Job Description"][i]
    x = txt.split()
    new_w=get_neigbours(x,find_words(x,words))
    req.append(new_w)
    
df = ugly_list_to_cute_df()
df.to_csv("both_neigbours.csv")


In [35]:
from nltk.corpus import wordnet


def get_synonims(word):
    synonyms = []
    for syn in wordnet.synsets(word):
            for l in syn.lemmas():
                synonyms.append(l.name())
    return set(synonyms)



words_extended = set()
for i in words:
    words_extended=words_extended.union(get_synonims(i))

In [37]:
len(words_extended)

3691